# KBG Syndrome

[KBG syndrome (KBGS)](https://omim.org/entry/148050) is caused by heterozygous mutation in the ANKRD11 gene. In this notebook, we have used
[pyphetools](https://github.com/monarch-initiative/pyphetools) to parse the clinical data included in the supplemental files of
[Martinez-Cayuelas E, et al. Clinical description, molecular delineation and genotype-phenotype correlation in 340 patients with KBG syndrome: addition of 67 new patients](https://pubmed.ncbi.nlm.nih.gov/36446582).

The authors identified a significantly higher frequency of patients with a triangular face in carriers of sequence variants compared to CNVs. Other associations found were short stature and variants in exon 9, a lower incidence of ID/ADHD/ASD in carriers of the c.1903_1907del variant and the size of the deletion, in CNV carriers, with the presence of macrodontia and hand anomalies.

In [1]:
import genophenocorr
import hpotk
from IPython.display import display, HTML
store = hpotk.configure_ontology_store()
hpo = store.load_minimal_hpo(release='v2023-10-09')
print(f'Loaded HPO v{hpo.version}')
print(f"Using genophenocorr version {genophenocorr.__version__}")

/Users/robin/PycharmProjects/genophenocorr/gpc_venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Loaded HPO v2023-10-09
Using genophenocorr version 0.1.1dev


## Settings
Specify the transcript to be used to encode the variants (the phenopackets contain VCF representations of small variants).
## TODO -- put all other settings here.

In [ ]:
ANKRD11_id = 'NM_013275.6' # MANE select

### Ingest all phenopackets in the cohort

In [4]:
from genophenocorr.preprocessing import configure_caching_cohort_creator, load_phenopacket_folder

fpath_phenopackets = 'phenopackets'
cohort_creator = configure_caching_cohort_creator(hpo, timeout=20)
cohort = load_phenopacket_folder(fpath_phenopackets, cohort_creator)



Patients Created: 100%|██████████| 337/337 [00:00<00:00, 473.06it/s]
Validated under none policy
Showing errors and warnings
337 phenopacket(s) found at `phenopackets`
  patient #0
    variants
     warnings:
     ·Expected a VCF record, a VRS CNV, or an expression with `hgvs.c` but had an error retrieving any from patient Novara, 2017_P2[PMID_36446582_Novara_2017_P2]. Remove variant from testing
     ·Patient PMID_36446582_Novara_2017_P2 has no variants to work with
  patient #1
    variants
     warnings:
     ·Expected a VCF record, a VRS CNV, or an expression with `hgvs.c` but had an error retrieving any from patient Goldenberg2016_P13[PMID_36446582_Goldenberg2016_P13]. Remove variant from testing
     ·Patient PMID_36446582_Goldenberg2016_P13 has no variants to work with
  patient #3
    variants
     warnings:
     ·Expected a VCF record, a VRS CNV, or an expression with `hgvs.c` but had an error retrieving any from patient Ockeloen2015_P20[PMID_36446582_Ockeloen2015_P20]. Remove

### Load Phenopackets

We will load phenopacket JSON files located in `phenopackets` folder that is next to the notebook.

In [5]:
from genophenocorr.view import CohortViewable
cv = CohortViewable(cohort=cohort, hpo_ontology=hpo, transcript_id=ANKRD11_id)
html = cv.get_html()
display(HTML(html))

HPO Term,ID,Annotation Count
Macrodontia,HP:0001572,170
Intellectual disability,HP:0001249,158
Abnormality of the hand,HP:0001155,155
Global developmental delay,HP:0001263,132
Delayed speech and language development,HP:0000750,121
Short stature,HP:0004322,115
Thick eyebrow,HP:0000574,106
Long philtrum,HP:0000343,104
Bulbous nose,HP:0000414,74
Triangular face,HP:0000325,68


### Pick transcript

We choose the [MANE Select](https://www.ncbi.nlm.nih.gov/nuccore/NM_013275.6) transcript for *ANKRD11*.

In [6]:
tx_id = 'NM_013275.6'

## Configure the analysis

In [7]:
from genophenocorr.analysis import configure_cohort_analysis, CohortAnalysisConfiguration
from genophenocorr.analysis.predicate import PatientCategories

analysis_config = CohortAnalysisConfiguration()
analysis_config.missing_implies_excluded = True
analysis_config.pval_correction = 'fdr_bh'
analysis_config.min_perc_patients_w_hpo = 0.1
analysis = configure_cohort_analysis(cohort, hpo, config=analysis_config)

Test for presence of genotype-phenotype correlations between frameshift variants vs. others.

In [9]:
from genophenocorr.model import VariantEffect

frameshift = analysis.compare_by_variant_effect(VariantEffect.FRAMESHIFT_VARIANT, ANKRD11_id)
frameshift.summarize(hpo, PatientCategories.YES)

FRAMESHIFT_VARIANT on NM_013275.6                       Yes             No  \
                                                      Count Percent  Count   
Abnormality of the hand [HP:0001155]                 95/144     66%  60/71   
EEG abnormality [HP:0002353]                           7/33     21%   9/16   
Feeding difficulties [HP:0011968]                     33/89     37%  26/45   
Low anterior hairline [HP:0000294]                    40/58     69%  15/30   
Intellectual disability [HP:0001249]                 99/119     83%  59/64   
...                                                     ...     ...    ...   
Aplasia/Hypoplasia of the mandible [HP:0009118]       13/13    100%    3/3   
Abnormal nervous system electrophysiology [HP:0...      7/7    100%    9/9   
Phenotypic abnormality [HP:0000118]                 174/174    100%  82/82   
Abnormal morphology of the nasal alae [HP:0000429]    41/41    100%  22/22   
Abnormality of the genital system [HP:0000078]        20/20    100%  15/15   

FRAMESHIFT_VARIANT on NM_013275.6                                     \
                                                   Percent   p value   
Abnormality of the hand [HP:0001155]                   85%  0.005661   
EEG abnormality [HP:0002353]                           56%  0.022884   
Feeding difficulties [HP:0011968]                      58%  0.027584   
Low anterior hairline [HP:0000294]                     50%  0.105274   
Intellectual disability [HP:0001249]                   92%  0.115195   
...                                                    ...       ...   
Aplasia/Hypoplasia of the mandible [HP:0009118]       100%  1.000000   
Abnormal nervous system electrophysiology [HP:0...    100%  1.000000   
Phenotypic abnormality [HP:0000118]                   100%  1.000000   
Abnormal morphology of the nasal alae [HP:0000429]    100%  1.000000   
Abnormality of the genital system [HP:0000078]        100%  1.000000   

FRAMESHIFT_VARIANT on NM_013275.6                                     
                                                   Corrected p value  
Abnormality of the hand [HP:0001155]                             1.0  
EEG abnormality [HP:0002353]                                     1.0  
Feeding difficulties [HP:0011968]                                1.0  
Low anterior hairline [HP:0000294]                               1.0  
Intellectual disability [HP:0001249]                             1.0  
...                                                              ...  
Aplasia/Hypoplasia of the mandible [HP:0009118]                  1.0  
Abnormal nervous system electrophysiology [HP:0...               1.0  
Phenotypic abnormality [HP:0000118]                              1.0  
Abnormal morphology of the nasal alae [HP:0000429]               1.0  
Abnormality of the genital system [HP:0000078]                   1.0  

[190 rows x 6 columns]

Test for presence of genotype-phenotype correlations between subjects with >=1 allele of a variant vs. the other subjects:


In [10]:
var_single = analysis.compare_by_variant_key('16_89284634_89284639_GTGTTT_G')
var_single.summarize(hpo, PatientCategories.YES)

>=1 allele of the variant 16_89284634_89284639_GTGTTT_G    Yes          \
                                                         Count Percent   
Sleep abnormality [HP:0002360]                             7/9     78%   
Autistic behavior [HP:0000729]                             1/6     17%   
Intellectual disability [HP:0001249]                     14/19     74%   
Synophrys [HP:0000664]                                    5/14     36%   
EEG abnormality [HP:0002353]                               0/6      0%   
...                                                        ...     ...   
Aplasia/Hypoplasia of the mandible [HP:0009118]            2/2    100%   
Abnormal nervous system electrophysiology [HP:0...         0/0      0%   
Phenotypic abnormality [HP:0000118]                      34/34    100%   
Abnormal morphology of the nasal alae [HP:0000429]         8/8    100%   
Abnormality of the genital system [HP:0000078]             5/5    100%   

>=1 allele of the variant 16_89284634_89284639_GTGTTT_G       No          \
                                                           Count Percent   
Sleep abnormality [HP:0002360]                             12/48     25%   
Autistic behavior [HP:0000729]                             39/66     59%   
Intellectual disability [HP:0001249]                     144/164     88%   
Synophrys [HP:0000664]                                     48/82     59%   
EEG abnormality [HP:0002353]                               16/43     37%   
...                                                          ...     ...   
Aplasia/Hypoplasia of the mandible [HP:0009118]            14/14    100%   
Abnormal nervous system electrophysiology [HP:0...         16/16    100%   
Phenotypic abnormality [HP:0000118]                      222/222    100%   
Abnormal morphology of the nasal alae [HP:0000429]         55/55    100%   
Abnormality of the genital system [HP:0000078]             30/30    100%   

>=1 allele of the variant 16_89284634_89284639_GTGTTT_G            \
                                                          p value   
Sleep abnormality [HP:0002360]                           0.004267   
Autistic behavior [HP:0000729]                           0.081923   
Intellectual disability [HP:0001249]                     0.147465   
Synophrys [HP:0000664]                                   0.148677   
EEG abnormality [HP:0002353]                             0.158804   
...                                                           ...   
Aplasia/Hypoplasia of the mandible [HP:0009118]          1.000000   
Abnormal nervous system electrophysiology [HP:0...       1.000000   
Phenotypic abnormality [HP:0000118]                      1.000000   
Abnormal morphology of the nasal alae [HP:0000429]       1.000000   
Abnormality of the genital system [HP:0000078]           1.000000   

>=1 allele of the variant 16_89284634_89284639_GTGTTT_G                    
                                                        Corrected p value  
Sleep abnormality [HP:0002360]                                   0.810716  
Autistic behavior [HP:0000729]                                   1.000000  
Intellectual disability [HP:0001249]                             1.000000  
Synophrys [HP:0000664]                                           1.000000  
EEG abnormality [HP:0002353]                                     1.000000  
...                                                                   ...  
Aplasia/Hypoplasia of the mandible [HP:0009118]                  1.000000  
Abnormal nervous system electrophysiology [HP:0...               1.000000  
Phenotypic abnormality [HP:0000118]                              1.000000  
Abnormal morphology of the nasal alae [HP:0000429]               1.000000  
Abnormality of the genital system [HP:0000078]                   1.000000  

[190 rows x 6 columns]

Or between subjects with one variant vs. the other variant.

In [11]:
var_double = analysis.compare_by_variant_keys('16_89284129_89284134_CTTTTT_C', '16_89284634_89284639_GTGTTT_G')
var_double.summarize(hpo, PatientCategories.YES)

>=1 allele of either variant 16_89284129_89284134_CTTTTT_C or variant 16_89284634_89284639_GTGTTT_G  First  \
                                                                                                     Count   
Autistic behavior [HP:0000729]                                                                         2/2   
Macrodontia [HP:0001572]                                                                               5/9   
Hypertelorism [HP:0000316]                                                                             3/3   
Cryptorchidism [HP:0000028]                                                                            0/4   
Low anterior hairline [HP:0000294]                                                                     3/3   
...                                                                                                    ...   
Aplasia/Hypoplasia of the mandible [HP:0009118]                                                        0/0   
Abnormal nervous system electrophysiology [HP:0...                                                     0/0   
Phenotypic abnormality [HP:0000118]                                                                  10/10   
Abnormal morphology of the nasal alae [HP:0000429]                                                     4/4   
Abnormality of the genital system [HP:0000078]                                                         0/0   

>=1 allele of either variant 16_89284129_89284134_CTTTTT_C or variant 16_89284634_89284639_GTGTTT_G          \
                                                                                                    Percent   
Autistic behavior [HP:0000729]                                                                         100%   
Macrodontia [HP:0001572]                                                                                56%   
Hypertelorism [HP:0000316]                                                                             100%   
Cryptorchidism [HP:0000028]                                                                              0%   
Low anterior hairline [HP:0000294]                                                                     100%   
...                                                                                                     ...   
Aplasia/Hypoplasia of the mandible [HP:0009118]                                                          0%   
Abnormal nervous system electrophysiology [HP:0...                                                       0%   
Phenotypic abnormality [HP:0000118]                                                                    100%   
Abnormal morphology of the nasal alae [HP:0000429]                                                     100%   
Abnormality of the genital system [HP:0000078]                                                           0%   

>=1 allele of either variant 16_89284129_89284134_CTTTTT_C or variant 16_89284634_89284639_GTGTTT_G Second  \
                                                                                                     Count   
Autistic behavior [HP:0000729]                                                                         1/6   
Macrodontia [HP:0001572]                                                                             22/27   
Hypertelorism [HP:0000316]                                                                            5/10   
Cryptorchidism [HP:0000028]                                                                           5/12   
Low anterior hairline [HP:0000294]                                                                    7/13   
...                                                                                                    ...   
Aplasia/Hypoplasia of the mandible [HP:0009118]                                                        2/2   
Abnormal nervous system electrophysiology [HP:0...                                                     0/0   
Phenotypic abnormality [H

TODO - finalize!